# **Imports**

In [1]:
import pandas as pd

# **Reading Data**

In [2]:
df = pd.read_csv('../Data/combined_data.csv')

# **Data Exploration and Analysis**

In [3]:
df.head()

,Latitude,Longitude,Altitude,Distance,Alignment,Distance_Sensors,Proximity,Wheel_Speed,Velocity,Hall_Effect,Emergency_Stop,Temperature,Humidity,Light_Level
0,37.775375,-121.897747,73.071218,0.0000,True,3.255424,False,4.750051,1.425015,True,False,22.780391,51.359185,576.224848
1,37.775375,-121.897746,73.071218,0.0003,True,2.279579,False,2.195034,0.658510,True,True,10.547808,33.711211,935.633297
2,37.775375,-121.897745,73.071218,0.0006,True,2.172817,True,3.777403,1.133221,False,True,13.817581,49.588575,654.387359
3,37.775375,-121.897744,73.071218,0.0009,True,1.358142,False,1.186756,0.356027,True,False,19.249456,32.246666,159.293949
4,37.775375,-121.897743,73.071218,0.0012,False,0.579548,True,2.024538,0.607362,True,True,11.694243,63.618960,442.111798


In [4]:
df.shape

(50000, 14)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Latitude          50000 non-null  float64
 1   Longitude         50000 non-null  float64
 2   Altitude          50000 non-null  float64
 3   Distance          50000 non-null  float64
 4   Alignment         50000 non-null  bool   
 5   Distance_Sensors  50000 non-null  float64
 6   Proximity         50000 non-null  bool   
 7   Wheel_Speed       50000 non-null  float64
 8   Velocity          50000 non-null  float64
 9   Hall_Effect       50000 non-null  bool   
 10  Emergency_Stop    50000 non-null  bool   
 11  Temperature       50000 non-null  float64
 12  Humidity          50000 non-null  float64
 13  Light_Level       50000 non-null  float64
dtypes: bool(4), float64(10)
memory usage: 4.0 MB


In [6]:
df.describe()

,Latitude,Longitude,Altitude,Distance,Distance_Sensors,Wheel_Speed,Velocity,Temperature,Humidity,Light_Level
count,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000,50000.000000
mean,37.517701,-121.497456,47.339932,0.149850,2.247534,2.751998,0.825600,22.504581,49.975236,500.511602
std,0.271785,0.290099,30.564202,0.086603,1.012972,1.298840,0.389652,7.222354,11.521304,289.707753
min,37.004829,-121.978642,0.265895,0.000000,0.500005,0.500162,0.150049,10.000035,30.000116,0.004272
25%,37.300933,-121.745357,18.167668,0.074925,1.368943,1.626524,0.487957,16.237724,39.933769,248.501353
50%,37.511985,-121.496175,43.877131,0.149850,2.245186,2.749673,0.824902,22.540286,50.028937,500.215745
75%,37.744905,-121.240882,80.082263,0.224775,3.131072,3.873745,1.162123,28.746879,59.893651,751.693333
max,38.000110,-121.002638,99.998455,0.299700,3.999987,4.999683,1.499905,34.998557,69.998334,999.908330


In [7]:
df.isna().sum()

Latitude            0
Longitude           0
Altitude            0
Distance            0
Alignment           0
Distance_Sensors    0
Proximity           0
Wheel_Speed         0
Velocity            0
Hall_Effect         0
Emergency_Stop      0
Temperature         0
Humidity            0
Light_Level         0
dtype: int64

In [8]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level'],
      dtype='object')

In [9]:
alignment_conditions = [
    alignment == True for alignment in df["Alignment"]
]

# Adding labels for Chaining Alignment
df["Alignment_Label"] = ["Aligned" if condition else "Not_Aligned" for condition in alignment_conditions]


In [10]:
# Define conditions for safety based on Hall_Effect and Emergency_Stop
safety_conditions = [
    hall_effect == True and emergency_stop == False
    for hall_effect, emergency_stop in zip(
        df["Hall_Effect"],
        df["Emergency_Stop"]
    )
]

# Adding labels for Safety Decision
df["Safety_Label"] = ["Safe" if condition else "Unsafe" for condition in safety_conditions]

In [11]:
# Speed thresholds (in meters per second - m/s)
threshold_high_speed = 3.5  

# Distance thresholds (in meter)
threshold_low_distance = 1

# Temperature thresholds (in degrees Celsius)
threshold_high_temperature = 30 

# Light level thresholds (in Lux)
threshold_low_light = 100


collision_risk_levels = []

for distance, proximity, wheel_speed, temperature, light_level in zip(
    df["Distance_Sensors"],
    df["Proximity"],
    df["Wheel_Speed"],
    df["Temperature"],
    df["Light_Level"]
):
    if proximity or wheel_speed > threshold_high_speed:
        collision_risk_levels.append(2)
    elif distance < threshold_low_distance:
        collision_risk_levels.append(1)
    elif temperature > threshold_high_temperature or light_level < threshold_low_light:
        collision_risk_levels.append(1)
    else:
        collision_risk_levels.append(0)


# Adding labels for Collision Avoidance
df["Collision_Avoidance_Label"] = ["Low_Risk" if risk_level == 0 else "Medium_Risk" if risk_level == 1 else "High_Risk" for risk_level in collision_risk_levels]

In [12]:
# Define acceptable distance thresholds for chaining 
max_distance = 3 
min_distance = 0.5

# Cart wheel speed treshold in meters per second (m/s) for chaining 
high_speed = 3.5

# Define conditions for chaining based on the data
chaining_conditions = [
    alignment and distance > min_distance and distance < max_distance and not proximity and wheel_speed < high_speed and safety_label == "Safe"
    for alignment, distance, proximity, safety_label, wheel_speed in zip(
        df["Alignment"],
        df["Distance_Sensors"],
        df["Proximity"],
        df["Safety_Label"],
        df['Wheel_Speed']
    )
]

# Adding labels for Chaining/Unchaining Decision
df["Chaining_Unchaining_Label"] = ["Chaining" if condition else "Unchaining" for condition in chaining_conditions]

In [13]:
# Save the updated combined data with labels to a new CSV file
df.to_csv("../Data/final_combined_data.csv", index=False)

print("Combined data with labels saved.")

Combined data with labels saved.


In [14]:
df.columns

Index(['Latitude', 'Longitude', 'Altitude', 'Distance', 'Alignment',
       'Distance_Sensors', 'Proximity', 'Wheel_Speed', 'Velocity',
       'Hall_Effect', 'Emergency_Stop', 'Temperature', 'Humidity',
       'Light_Level', 'Alignment_Label', 'Safety_Label',
       'Collision_Avoidance_Label', 'Chaining_Unchaining_Label'],
      dtype='object')

In [15]:
df.isna().sum()

Latitude                     0
Longitude                    0
Altitude                     0
Distance                     0
Alignment                    0
Distance_Sensors             0
Proximity                    0
Wheel_Speed                  0
Velocity                     0
Hall_Effect                  0
Emergency_Stop               0
Temperature                  0
Humidity                     0
Light_Level                  0
Alignment_Label              0
Safety_Label                 0
Collision_Avoidance_Label    0
Chaining_Unchaining_Label    0
dtype: int64